In [45]:
import sys
import os
import time
sys.path.append(os.path.abspath(os.path.dirname(os.getcwd())))
sys.path.append(os.path.abspath(os.getcwd()))

import casadi as cd
from Envs.robot_arm import EFFECTOR_env_mj, End_Effector_model
from Solvers.OCsolver import  ocsolver_v4
import numpy as np
from matplotlib import pyplot as plt

from utils.Visualize_mj import arm_visualizer_mj_v1
from scipy.spatial.transform import Rotation as R
import mujoco

In [46]:
dt=0.1
Horizon=10
target_end_pos=[0.3,0.4,0.5]
target_quat=[0,1,0,0]
target_x=target_end_pos+target_quat

arm_model=End_Effector_model(dt=dt)
dyn_f = arm_model.get_dyn_f()

step_cost_vec = np.array([1.0,0.5,0.4,0.3]) * 1e-1
step_cost_f = arm_model.get_step_cost_param(step_cost_vec)
term_cost_vec = np.array([1.0,0.5]) * 1e0
term_cost_f = arm_model.get_terminal_cost_param(term_cost_vec)

controller = ocsolver_v4('arm control')
controller.set_state_param(7, None, None)
controller.set_ctrl_param(6, 6*[-1e10], 6*[1e10])
controller.set_dyn(dyn_f)
controller.set_step_cost(step_cost_f)
controller.set_term_cost(term_cost_f)
controller.construct_prob(horizon=Horizon)

In [47]:
test_p=np.array([0.3,0.2,0.4]).reshape(-1,1)
test_quat=np.array([0.0,0.707,0.707,0]).reshape(-1,1)
test_quat/=np.linalg.norm(test_quat)
test_x=np.concatenate([test_p,test_quat])

u=controller.control(test_x,target_x=target_x)

In [48]:
print(u)
traj=controller.opt_traj
print(traj[-7:])

[-1.57285262e-18  2.97626508e-01  1.48813254e-01  0.00000000e+00
  0.00000000e+00 -1.40001919e+00]
[0.3        0.36671162 0.48335581 0.         0.96609797 0.30308885
 0.        ]
